### `Parte_1: Limpieza de datos de Alumnos Prioritarios, Preferentes y/o Beneficiarios ` 

trabajaremos en limpiar esta bases de datos:

* Esquema de registro Alumnos Prioritarios, Preferentes y/o Beneficiarios de la Subvención 


En este notebook se relizará la limpieza de los datos sobre el resumen de rendimiento academico y Adicionalmente, se creará un archivo con los datos limpios convinados entre as bases de datos previamente limpios:



In [3]:
# importacion de librerias 
import os 
import pandas as pd 

In [16]:
# Definir la ruta del archivo ' abrevie Preferentes_Prioritarios_y_Beneficiarios como PPB y Resumen_Rendimiento como RR 
ruta_PPB = os.path.join('data', 'Preferentes_Prioritarios_y_Beneficiarios_2022.csv')
ruta_Asistencia= os.path.join('output_data', 'asistencia.csv')
ruta_rendimiento= os.path.join('output_data', 'rendimiento_estudiantes.csv')


### `Verificar valores atípicos y Renombrar datos de Preferentes_Prioritarios_y_Beneficiarios.`

* las columnas a tomar en cuenta se describen a continuación

| Columna          | Descripción                                             |
|------------------|---------------------------------------------------------|
| MRUN             | Identificador único del estudiante (máscara del RUN)     |
| PRIORITARIO_ALU  | Indicador de alumno prioritario (0: No prioritario, 1: prioritario)           |
| PREFERENTE_ALU   | Indicador de alumno preferente (0: No preferente, 1: Preferente) |
| BEN_SEP          | Indicador de alumno beneficiario de la SEP (0: No beneficiario SEP, 1: Beneficiario SEP ) |
| CRITERIO_SEP     | Criterio por el cual se considera prioritario: <br>0: No prioritario (No cumple requisito o Puntaje Alto)<br>1: Pertenece a Chile Solidario<br>2: Puntaje Ficha de Protección Social menor o igual al punto de corte<br>3: Otro criterio |



In [5]:
# dataframe alumnos Preferentes_Prioritarios_y_Beneficiarios
df_PPB = pd.read_csv(ruta_PPB, sep=';')

df_PPB = df_PPB[['MRUN', 'CRITERIO_SEP', 'BEN_SEP', 'PREFERENTE_ALU', 'PRIORITARIO_ALU','COD_REG_RBD','COD_COM_RBD','RURAL_RBD']]

/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_23853/2018955562.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_PPB = pd.read_csv(ruta_PPB, sep=';')


In [7]:
# aplicamos una funcion que verifica que los datos de las columna seleccionada no difiera de 1 o 0

def verificar_valores_atipicos(df, column):
    '''
    Retorna True si los valores difieren de 0 o 1 y False en caso contrario 
    
    '''
    hay_valor_distinto = (df[column] != 0) & (df[column] != 1)
    resultado = hay_valor_distinto.any()
    return resultado

print(verificar_valores_atipicos(df_PPB,'BEN_SEP'))

print(verificar_valores_atipicos(df_PPB, 'PREFERENTE_ALU' ))

print(verificar_valores_atipicos(df_PPB,'PRIORITARIO_ALU'))


False
False
False


In [8]:
# En estas lineas trataremos la legibilidad de nuestros datos, modificando nombres de variables 

# Reemplazar los valores en la columna 'PRIORITARIO_ALU'
df_PPB['PRIORITARIO_ALU'] = df_PPB['PRIORITARIO_ALU'].replace({0: 'No prioritario', 1: 'prioritario'})
# Reemplazar los valores en la columna 'PREFERENTE_ALU'
df_PPB['PREFERENTE_ALU'] = df_PPB['PREFERENTE_ALU'].replace({0: 'No preferente', 1: 'Preferente'})
# Reemplazar los valores en la columna 'BEN_SEP'
df_PPB['BEN_SEP'] = df_PPB['BEN_SEP'].replace({0: 'No beneficiario SEP', 1: 'Beneficiario SEP'})

In [9]:
df_PPB.head()

,RUN_ALUMNO,CRITERIO_SEP,BEN_SEP,PREFERENTE_ALU,PRIORITARIO_ALU,COD_REG_RBD,COD_COM_RBD,RURAL_RBD
0,25,0,No beneficiario SEP,Preferente,No prioritario,,,
1,31,1,Beneficiario SEP,No preferente,prioritario,13,13604,0
2,47,2,No beneficiario SEP,No preferente,prioritario,5,5502,0
3,48,2,Beneficiario SEP,No preferente,prioritario,6,6101,0
4,86,1,Beneficiario SEP,No preferente,prioritario,16,16107,0


### `Realizamos el merge con Asistencia.csv y Rendimiento.csv ( datos limpios )`

* realizar el merge por la columna 'RUN_ALUMNO'
* depositar el dataframe resultante e un archivo.csv

In [17]:
df_asistencia = pd.read_csv(ruta_Asistencia, sep=',')

df_rendimiento =  pd.read_csv(ruta_rendimiento, sep=',')

In [18]:
#renombramos la columna MRUN para mayor legebilidad 
df_rendimiento = df_rendimiento.rename(columns={'MRUN': 'RUN_ALUMNO'})

# Asegura que 'PROM_GRAL' no sea igual a 0 (Retirado)
df_rendimiento = df_rendimiento[df_rendimiento['PROM_GRAL'] != float(0)]

In [20]:
merged_df = pd.merge(df_asistencia, df_PPB, on='RUN_ALUMNO')

merged_df_estudiantes =  pd.merge(merged_df, df_rendimiento, on='RUN_ALUMNO')

merged_df_estudiantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 518029 entries, 0 to 518028
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   MES                      518029 non-null  int64  
 1   COD_PRO_RBD              518029 non-null  int64  
 2   COD_COM_RBD_x            518029 non-null  int64  
 3   RBD_x                    518029 non-null  int64  
 4   RUN_ALUMNO               518029 non-null  int64  
 5   GENERO_ALUMNO            518029 non-null  int64  
 6   FECHA_NACIMIENTO_ALUMNO  518029 non-null  int64  
 7   NOMBRE_COMUNA_ALUMNO     518029 non-null  object 
 8   COD_ENSE2                518029 non-null  int64  
 9   DIAS_ASISTIDOS           518029 non-null  int64  
 10  ASISTENCIA_PROMEDIO      518029 non-null  float64
 11  CRITERIO_SEP             518029 non-null  int64  
 12  BEN_SEP                  518029 non-null  object 
 13  PREFERENTE_ALU           518029 non-null  object 
 14  PRIO

In [22]:
merged_df_estudiantes

,MES,COD_PRO_RBD,COD_COM_RBD_x,RBD_x,RUN_ALUMNO,GENERO_ALUMNO,FECHA_NACIMIENTO_ALUMNO,NOMBRE_COMUNA_ALUMNO,COD_ENSE2,DIAS_ASISTIDOS,...,BEN_SEP,PREFERENTE_ALU,PRIORITARIO_ALU,COD_REG_RBD,COD_COM_RBD_y,RURAL_RBD,RBD_y,COD_JOR,PROM_GRAL,SIT_FIN_R
0,8,31,3101,396,2112520,2,200901,COPIAPÓ,7,14,...,Beneficiario SEP,No preferente,prioritario,3,3101,1,423,1,"6,2",P
1,8,31,3101,396,3962434,1,200901,COPIAPÓ,7,12,...,Beneficiario SEP,Preferente,No prioritario,3,3101,0,420,3,"4,9",P
2,8,31,3101,396,4199164,1,200707,COPIAPÓ,7,10,...,Beneficiario SEP,No preferente,prioritario,3,3101,0,403,1,"5,2",P
3,8,31,3101,396,4218504,2,200812,COPIAPÓ,7,15,...,Beneficiario SEP,No preferente,prioritario,3,3101,0,430,3,"6,6",P
4,8,31,3101,396,6990009,2,200903,COPIAPÓ,7,15,...,Beneficiario SEP,No preferente,prioritario,3,3101,0,430,3,"6,7",P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518024,8,131,13130,41794,25757929,2,201110,SAN MIGUEL,2,20,...,Beneficiario SEP,Preferente,No prioritario,13,13130,0,41794,1,"6,3",P
518025,8,131,13130,41794,26254392,1,201106,SAN MIGUEL,2,19,...,Beneficiario SEP,Preferente,No prioritario,13,13130,0,11906,2,"6,3",P
518026,8,131,13130,41794,26305268,2,201112,SAN MIGUEL,2,21,...,Beneficiario SEP,Preferente,No prioritario,13,13130,0,41794,1,"6,6",P
518027,8,131,13130,41794,26498431,1,201108,SANTIAGO,2,21,...,Beneficiario SEP,Preferente,No prioritario,13,13130,0,11813,1,0,T


In [23]:
merged_df_estudiantes.to_csv('output_data/info_por_estudiante.csv', index=False)